# WSDM - KKBox's Churn Prediction Challenge

Auteur: A. Carrere

Date: 15 décembre 2017

In [2]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import xgboost as xgb
from sklearn import *
import sklearn
print("---IMPORTS OK---")

/home/arnaud/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


---IMPORTS OK---


/home/arnaud/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/arnaud/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [ ]:
#Data loading
print("---LOADING DATA---")
train = pd.concat((pd.read_csv('../input/train.csv'), pd.read_csv('../input/train_v2.csv')), axis=0, ignore_index=True).reset_index(drop=True)
test = pd.read_csv('../input/sample_submission_v2.csv')
members = pd.read_csv('../input/members_v3.csv')
transactions = pd.concat((pd.read_csv('../input/transactions.csv'), pd.read_csv('../input/transactions_v2.csv')), axis=0, ignore_index=True).reset_index(drop=True)
transactions = transactions.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)
transactions = transactions.drop_duplicates(subset=['msno'], keep='first')
#df_iter = pd.read_csv('../input/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
#user_logs_v2 = pd.read_csv('../input/user_logs_v2.csv',nrows=100000)
print("---END---")

---LOADING DATA---


In [6]:
#Feature engineering
print("---FEATURE ENGINEERING---")
train['is_churn'] = train['is_churn'].astype(np.int8)

members['city'] = members['city'].astype(np.int8)
members['bd'] = members['bd'].astype(np.int16)
members['registered_via'] = members['registered_via'].astype(np.int8)
members['registration_init_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[:4]))
members['registration_init_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration_init_date'] = members['registration_init_time'].apply(lambda x: int(str(x)[-2:]))
members['registration_init_year'] = members['registration_init_year'].astype(np.int16)
members['registration_init_month'] = members['registration_init_month'].astype(np.int8)
members['registration_init_date'] = members['registration_init_date'].astype(np.int8)
gender = {'male':1, 'female':2}
members['gender'] = members['gender'].map(gender)

transactions['payment_method_id'] = transactions['payment_method_id'].astype(np.int8)
transactions['payment_plan_days'] = transactions['payment_plan_days'].astype(np.int16)
transactions['plan_list_price'] = transactions['plan_list_price'].astype(np.int16)
transactions['actual_amount_paid'] = transactions['actual_amount_paid'].astype(np.int16)
transactions['is_auto_renew'] = transactions['is_auto_renew'].astype(np.int8)
transactions['is_cancel'] = transactions['is_cancel'].astype(np.int8)
transactions['transaction_date_year'] = transactions['transaction_date'].apply(lambda x: int(str(x)[:4]))
transactions['transaction_date_month'] = transactions['transaction_date'].apply(lambda x: int(str(x)[4:6]))
transactions['transaction_date_date'] = transactions['transaction_date'].apply(lambda x: int(str(x)[-2:]))
transactions['transaction_date_year'] = transactions['transaction_date_year'].astype(np.int16)
transactions['transaction_date_month'] = transactions['transaction_date_month'].astype(np.int8)
transactions['transaction_date_date'] = transactions['transaction_date_date'].astype(np.int8)
transactions['membership_expire_date_year'] = transactions['membership_expire_date'].apply(lambda x: int(str(x)[:4]))
transactions['membership_expire_date_month'] = transactions['membership_expire_date'].apply(lambda x: int(str(x)[4:6]))
transactions['membership_expire_date_date'] = transactions['membership_expire_date'].apply(lambda x: int(str(x)[-2:]))
transactions['membership_expire_date_year'] = transactions['membership_expire_date_year'].astype(np.int16)
transactions['membership_expire_date_month'] = transactions['membership_expire_date_month'].astype(np.int8)
transactions['membership_expire_date_date'] = transactions['membership_expire_date_date'].astype(np.int8)
transactions['discount'] = transactions['plan_list_price'] - transactions['actual_amount_paid']
transactions['amt_per_day'] = transactions['actual_amount_paid'] / transactions['payment_plan_days']
transactions['is_discount'] = transactions.discount.apply(lambda x: 1 if x > 0 else 0)
transactions['membership_days'] = pd.to_datetime(transactions['membership_expire_date']).subtract(pd.to_datetime(transactions['transaction_date'])).dt.days.astype(int)
print("---END---")

---FEATURE ENGINEERING---
---END---


In [ ]:
#Multiprocessing to model user_logs
print("---USER_LOGS---")

def transform_df(df):
    df = pd.DataFrame(df)
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

def transform_df2(df):
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

last_user_logs = []
i = 0 #~400 Million Records - starting at the end but remove locally if needed
for df in df_iter:
    if i>35:
        if len(df)>0:
            #print(df.shape)
            p = Pool(cpu_count())
            df = p.map(transform_df, np.array_split(df, cpu_count()))   
            df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
            df = transform_df2(df)
            p.close(); p.join()
            last_user_logs.append(df)
            #print('...', df.shape)
            df = []
    i+=1

last_user_logs.append(transform_df(user_logs_v2))
last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = transform_df2(last_user_logs)
print("---END---")

---USER_LOGS---


In [3]:
#Datadets merging
print("---MERGING---")
#We concatenate train+test to merge with transactions and members
train['is_train'] = 1
test['is_train'] = 0
combined = pd.concat([train, test], axis = 0)
combined = pd.merge(combined, members, how='left', on='msno')
del members
combined = pd.merge(combined, transactions, how='left', on='msno')
del transactions
#We separate train and split after merging
train = combined[combined['is_train'] == 1]
test = combined[combined['is_train'] == 0]
train.drop(['is_train'], axis = 1, inplace = True)
test.drop(['is_train'], axis = 1, inplace = True)
del combined
#Merging of train and test with the user logs
#train = pd.merge(train, last_user_logs, how='left', on='msno')
#test = pd.merge(test, last_user_logs, how='left', on='msno')
#last_user_logs=[]
print("---END---")

---MERGING---


NameError: name 'train' is not defined

In [ ]:
#Last feature engineering
print("---CANCEL---")
train['autorenew_&_not_cancel'] = ((train.is_auto_renew == 1) == (train.is_cancel == 0)).astype(np.int8)
test['autorenew_&_not_cancel'] = ((test.is_auto_renew == 1) == (test.is_cancel == 0)).astype(np.int8)
train['notAutorenew_&_cancel'] = ((train.is_auto_renew == 0) == (train.is_cancel == 1)).astype(np.int8)
test['notAutorenew_&_cancel'] = ((test.is_auto_renew == 0) == (test.is_cancel == 1)).astype(np.int8)
train = train.fillna(0)
test = test.fillna(0)
print("---END---")

In [ ]:
#Testing with xgboost
print("---TESTING---")
cols = [c for c in train.columns if c not in ['is_churn','msno']]

def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', metrics.log_loss(labels, preds)

fold = 1
for i in range(fold):
    params = {
        'eta': 0.02, 
        'max_depth': 7,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': 100,
        'silent': True
    }
    x1, x2, y1, y2 = model_selection.train_test_split(train[cols], train['is_churn'], test_size=0.3, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 150,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=50) #use 1500
    if i != 0:
        pred1 += model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
    else:
        pred1 = model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
pred1 /= fold

test['is_churn'] = pred1.clip(0.+1e-15, 1-1e-15)
test[['msno','is_churn']].to_csv('../output/submission.csv', index=False)

print("---DONE---")